# Midterm Question 8 a

In [65]:
from skimage.feature import hog
from skimage.io import imread
from skimage.transform import rescale
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [66]:
from typing import Mapping
from sympy import ImageSet


includeImage = ['agri_0_9971.jpeg', 'agri_0_9963.jpeg', 'agri_0_9960.jpeg',
           'agri_0_9952.jpeg', 'agri_0_9933.jpeg', 'agri_0_9927.jpeg', 'agri_0_9916.jpeg', 'agri_0_9908.jpeg', 'agri_0_9899.jpeg']

path = "problem8/agri_data/data/"

ImageSet = []
MappingSet = ["weed", "weed", "weed", "crop", "weed", "weed", "crop", "crop", "weed"]

for file in includeImage:
    image = imread(path + file)
    ImageSet.append(np.array(image))
    # np.array([local_hog(img) for img in X])

# for file in includeMap:
#     MappingSet.append(pd.read_csv(path + file, sep = " "))

# ImageSet
# MappingSet

# for set in ImageSet:
    # print(set[0])
    # print(set[0].shape[0])

In [67]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    ImageSet, 
    MappingSet, 
    test_size=0.2, 
    shuffle=True,
    random_state=42,
)

In [68]:
from sklearn.base import BaseEstimator, TransformerMixin
 
class RGB2GrayTransformer(BaseEstimator, TransformerMixin):
    """
    Convert an array of RGB images to grayscale
    """
 
    def __init__(self):
        pass
 
    def fit(self, X, y=None):
        """returns itself"""
        return self
 
    def transform(self, X, y=None):
        """perform the transformation and return an array"""
        return np.array([skimage.color.rgb2gray(img) for img in X])
     
 
class HogTransformer(BaseEstimator, TransformerMixin):
    """
    Expects an array of 2d arrays (1 channel images)
    Calculates hog features for each img
    """
 
    def __init__(self, y=None, orientations=9,
                 pixels_per_cell=(8, 8),
                 cells_per_block=(3, 3), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
 
        def local_hog(X):
            return hog(X,
                       orientations=self.orientations,
                       pixels_per_cell=self.pixels_per_cell,
                       cells_per_block=self.cells_per_block,
                       block_norm=self.block_norm)
 
        try: # parallel
            return np.array([local_hog(img) for img in X])
        except:
            return np.array([local_hog(img) for img in X])

In [69]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler, Normalizer
import skimage

# create an instance of each transformer
grayify = RGB2GrayTransformer()
hogify = HogTransformer(
    pixels_per_cell=(14, 14), 
    cells_per_block=(2,2), 
    orientations=9, 
    block_norm='L2-Hys'
)
scalify = StandardScaler()

X_train_gray = grayify.fit_transform(X_train)
X_train_hog = hogify.fit_transform(X_train_gray)
X_train_prepared = scalify.fit_transform(X_train_hog)

X_test_gray = grayify.fit_transform(X_test)
X_test_hog = hogify.fit_transform(X_test_gray)
X_test_prepared = scalify.fit_transform(X_test_hog)
 
print(X_train_prepared.shape)

(7, 44100)


In [70]:
sgd_clf = SGDClassifier(random_state=42, max_iter=1000, tol=1e-3)
sgd_clf.fit(X_train_prepared, y_train)
sgd_clf.score(X_test_prepared , y_test)

1.0